In [1]:
import pandas as pd
import pickle

import re
from tqdm.notebook import tqdm

In [9]:
from openai import OpenAI
client = OpenAI()

def embed(batch):
    response = client.embeddings.create(
        input=batch,
        model="text-embedding-3-small",
        dimensions=1024
    )
    return [data.embedding for data in response.data]


In [ ]:
data = pd.read_csv("./geodata.csv")
data = data[data["lang"] == "en"]
data

In [ ]:
def clean_text(df, field):
    df[field] = df[field].str.lower()
    df[field] = df[field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t$.!?,\"\':#%&])|(\w+:\/\/\S+)|^rt|http.+?", "", str(elem))) 
    return df

data = clean_text(data, 'text')
data

In [ ]:
data['time'] = pd.to_datetime(data[['year','month','day','hour']])
data = data.drop(columns=["year", "month", "day", "hour", "lang"])
data

In [ ]:
def batch(it, n=1):
    l = len(it)
    for ndx in range(0, l, n):
        yield it[ndx:min(ndx + n, l)]

def addEmbeddings(df):
    out = []
    for batches in tqdm(list(batch(df["text"],1000))):
        [out.append(x) for x in embed(batches)]
    
    df["embed"] = out
    

addEmbeddings(data)
data
    

In [11]:
with open("geodata.pickle", "wb") as f:
    pickle.dump(data, f)